<a href="https://colab.research.google.com/github/LethargicDemigod/LOL-drafter/blob/main/LOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install tqdm

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm


In [2]:
import csv
import torch

def load_champion_vectors(csv_path):
    champ_to_vec = {}
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["champion"]
            vec = torch.tensor([
                float(row["damage"]),
                float(row["toughness"]),
                float(row["control"]),
                float(row["mobility"]),
                float(row["utility"])
            ], dtype=torch.float32)
            champ_to_vec[name] = vec
    return champ_to_vec



In [3]:
import json

def match_loader(jsonl_path,start=0,end=None):
    with open(jsonl_path, 'r') as f:
        for idx, line in enumerate(f):
            if idx < start:
                continue
            if end is not None and idx >= end:
                break
            try:
                match = json.loads(line)
                yield idx, match
            except json.JSONDecodeError:
                print(f"Skipping line-{idx}")
                continue



In [4]:
class ChampModel(nn.Module):
    def __init__(self, champion_vectors, learnable=False):
        super().__init__()
        self.champ_names = list(champion_vectors.keys())
        initial_vecs = torch.stack([champion_vectors[name] for name in self.champ_names])
        self.embedding = nn.Embedding(171,25)
        nn.init.xavier_uniform_(self.embedding.weight)
        self.name_to_idx = {name: i for i, name in enumerate(self.champ_names)}

        self.attn1 = nn.MultiheadAttention(embed_dim=25, num_heads=5, batch_first=True)
        self.norm1=nn.LayerNorm(25)
        self.mlp1= nn.Sequential(
            nn.Linear(25, 25),
            nn.ReLU(),
            nn.Linear(25, 25)
        )
        self.norm2=nn.LayerNorm(25)
        self.attn2 = nn.MultiheadAttention(embed_dim=25, num_heads=5, batch_first=True)
        self.norm3=nn.LayerNorm(25)
        self.mlp2= nn.Sequential(
            nn.Linear(25, 25),
            nn.ReLU(),
            nn.Linear(25, 25)
        )
        self.norm4=nn.LayerNorm(25)

        self.output = nn.Linear(25, 1)

    def freeze_embeddings(self):
        self.embedding.weight.requires_grad = False

    def unfreeze_embeddings(self):
        self.embedding.weight.requires_grad = True

    def forward(self, batch_teams):
      batch_idxs = [
        [self.name_to_idx[name] for name in team]
        for team in batch_teams
      ]

      idx_tensor = torch.tensor(batch_idxs, device=self.embedding.weight.device)
      vectors = self.embedding(idx_tensor)

      attn_out1, _ = self.attn1(vectors, vectors, vectors)
      x=self.mlp1(attn_out1)
      attn_out2, _ = self.attn2(x, x, x)
      x=self.mlp2(attn_out2)
      pooled = attn_out1.mean(dim=1)
      return self.output(pooled).squeeze(-1)
    def freeze_attention(self):
      for param in self.attn1.parameters():
        param.requires_grad = False
      for param in self.attn2.parameters():
        param.requires_grad = False

    def unfreeze_attention(self):
      for param in self.attn1.parameters():
        param.requires_grad = True
      for param in self.attn2.parameters():
        param.requires_grad = True


In [5]:
from tqdm import tqdm

def train_phase(model, start, end, jsonl_path, optimizer, batch_size=32):
    batch_blue, batch_red, batch_labels = [], [], []

    total = end - start
    iterator = tqdm(match_loader(jsonl_path, start, end), total=total, desc="Training")

    for idx, match in iterator:
        blue = [p["championName"] for p in match["blueTeam"]]
        red = [p["championName"] for p in match["redTeam"]]
        winner = match["winningTeam"]

        batch_blue.append(blue)
        batch_red.append(red)
        batch_labels.append(0 if winner == 100 else 1)

        if len(batch_blue) == batch_size:
            model.train()
            optimizer.zero_grad()

            blue_scores = model(batch_blue)
            red_scores = model(batch_red)

            logits = torch.sigmoid(blue_scores-red_scores)
            labels = torch.tensor(batch_labels, dtype=torch.float32, device=logits.device)

            loss = F.binary_cross_entropy_with_logits(logits,labels)
            loss.backward()
            optimizer.step()


            iterator.set_postfix(loss=loss.item())

            batch_blue, batch_red, batch_labels = [], [], []




In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [7]:
from google.colab import drive
drive.mount('/content/drive')
jsonl=jsonl_path = "/content/drive/My Drive/deduplicated_matches.jsonl"
vector_embeddings="/content/drive/My Drive/champion_playstyle_vectors.csv"


Mounted at /content/drive


In [8]:
import torch , random
champion_vectors = load_champion_vectors(vector_embeddings)
champion_names = list(champion_vectors.keys())
new_champ_vectors={}
for champ in champion_names :
   new_champ_vectors[champ]=torch.rand(25)
cleaned_champ_vectors = {
    k: v for k, v in champion_vectors.items()
    if k is not None and k != "None"
}
print(len(cleaned_champ_vectors))

171


In [23]:


model = ChampModel(cleaned_champ_vectors, learnable=True).to(device)
for epochs in range(30):
  print(f"epoch{epochs}")
  model.freeze_attention()
  optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-2)

  train_phase(model, 0, 3_570_000, jsonl, optimizer,batch_size=32)

for n_epochs in range(60):
  print(f"epoch{n_epochs+5}")
  model.unfreeze_attention()
  optimizer=torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
  train_phase(model, 0, 3_570_000, jsonl, optimizer, batch_size=32)


epoch0


Training: 100%|██████████| 171000/171000 [01:02<00:00, 2740.68it/s, loss=0.685]


epoch1


Training: 100%|██████████| 171000/171000 [01:02<00:00, 2733.63it/s, loss=0.67]


epoch2


...

Training: 100%|██████████| 171000/171000 [01:06<00:00, 2555.84it/s, loss=0.722]


epoch23


Training: 100%|██████████| 171000/171000 [01:09<00:00, 2459.92it/s, loss=0.704]


epoch24


Training: 100%|██████████| 171000/171000 [01:07<00:00, 2516.80it/s, loss=0.704]


In [21]:
embeddings = model.embedding.weight.data.cpu().numpy()
idx_to_name = {i: name for name, i in model.name_to_idx.items()}

for i, vector in enumerate(embeddings):
    name = idx_to_name[i]
    print(f"{name}: {vector}")



Annie: [-0.9226638   2.2328565   1.4044354  -0.9282027   1.8227818   2.4394674
 -2.7681637  -0.37173468  3.8683543  -2.0054867   1.0954802  -0.23527133
  2.1538973  -2.1296608  -0.84010077  2.896468   -3.766542    1.2972755
  1.0595247  -1.7364905  -1.8591913  -0.15433612  2.193149   -1.7030712
 -2.3658614 ]
Olaf: [ 1.7292154  -2.2932682   0.5177614  -1.0394078  -3.9217901  -2.6070697
  3.587276    0.47967356  1.0045431  -0.3913138  -0.04576604  0.55376995
  0.00787961  1.608246   -1.3148997  -0.7754894   1.8027974  -0.6086022
 -5.849873    1.1034861   1.2468374   1.6073565   3.3041658  -2.5351908
  4.491555  ]
Galio: [ 3.409223   -3.4448993   0.7520576  -1.9100348  -0.62884027  3.0229049
  1.2549347   2.915588   -2.7925508  -1.1386472   3.0739322  -0.11655661
 -1.7208654  -0.7874046   0.3393953  -0.15283002  3.8377323  -0.3996694
 -0.9601866   2.4289198  -1.917128   -1.0113384  -0.03924594  2.8437512
  2.0903134 ]
TwistedFate: [-0.7660138  -0.3668051  -2.3430972   1.519925   -2.099785

In [20]:
print(model)

ChampModel(
  (embedding): Embedding(171, 25)
  (attn1): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=25, out_features=25, bias=True)
  )
  (norm1): LayerNorm((25,), eps=1e-05, elementwise_affine=True)
  (mlp1): Sequential(
    (0): Linear(in_features=25, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=25, bias=True)
  )
  (norm2): LayerNorm((25,), eps=1e-05, elementwise_affine=True)
  (attn2): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=25, out_features=25, bias=True)
  )
  (norm3): LayerNorm((25,), eps=1e-05, elementwise_affine=True)
  (mlp2): Sequential(
    (0): Linear(in_features=25, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=25, bias=True)
  )
  (norm4): LayerNorm((25,), eps=1e-05, elementwise_affine=True)
  (output): Linear(in_features=25, out_features=1, bias=True)
)


In [17]:
from torch.nn import functional as F
import torch

def evaluate(model, data, batch_size=32):
    model.eval()
    all_losses = []
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]

            batch_blue = [[champ['championName'] for champ in item['blueTeam']] for item in batch]
            batch_red = [[champ['championName'] for champ in item['redTeam']] for item in batch]
            labels = [1.0 if item['winningTeam'] == 'blue' else 0.0 for item in batch]

            blue_scores = model(batch_blue)
            red_scores = model(batch_red)
            preds = torch.sigmoid(blue_scores - red_scores)

            target = torch.tensor(labels, device=preds.device,dtype=torch.float32)
            loss = F.smooth_l1_loss(preds, target, beta=0)
            all_losses.append(loss.item())

            # Accuracy
            pred_labels = (preds > 0.5).float()
            correct += (pred_labels == target).sum().item()
            total += len(batch)


    avg_loss = sum(all_losses) / len(all_losses)
    accuracy = correct / total

    return avg_loss, accuracy


In [25]:
import json

def load_jsonl_range(path, start, end):
    with open(path, 'r') as f:
        lines = f.readlines()
    return [json.loads(line) for line in lines[start:end]]

eval_data = load_jsonl_range(jsonl, 3_570_000, None)



In [26]:
loss, acc = evaluate(model, eval_data)
print(f"Evaluation Loss: {loss:.4f}")
print(f"Accuracy: {acc*100:.2f}%")


Evaluation Loss: 0.4986
Accuracy: 50.19%
